#Ejemplo: generar texto de cuentos, usando Keras

#Instalando librerias CUDA

In [1]:
!nvcc --version  #Version de CUDA en la maquina virtual

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2020 NVIDIA Corporation
Built on Mon_Oct_12_20:09:46_PDT_2020
Cuda compilation tools, release 11.1, V11.1.105
Build cuda_11.1.TC455_06.29190527_0


##P0. importar librerias

In [2]:
import tensorflow as tf
import timeit               #para medir tiempos
import numpy as np
import pandas as pd 
import os
import time
import sys

#Uso de GPU para entrenar en tensorflow

In [3]:
print("Tensorflow Version: ", tf.__version__)
print("Dispositivos disponibles para entrenar: ", tf.config.list_physical_devices())
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Encontrada la GPU: {}'.format(device_name))

Tensorflow Version:  2.7.0
Dispositivos disponibles para entrenar:  [PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'), PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
Encontrada la GPU: /device:GPU:0


#Dejando activo la GPU

In [4]:
#tf.device('/gpu:0') #activando la CPU
tf.device('/device:GPU:0') #activando la GPU

##P0. Descarga y preprocesado de los datos

In [5]:
fileDL= tf.keras.utils.get_file('coran.txt','https://raw.githubusercontent.com/Julianjimenez98/DeepLearning/main/coran.txt')
texto = open(fileDL, 'rb').read().decode(encoding='utf-8-sig')

745472/736593 [==============================] - 0s 0us/step


##P1. entendiendo el texto

In [6]:
print('el texto tiene longitud de:{} caracteres'. format(len(texto)))
vocab = sorted(set(texto))
print('el texto esta compuesto de estos :{} caracteres'. format(len(vocab)))
print(vocab)

el texto tiene longitud de:735146 caracteres
el texto esta compuesto de estos :31 caracteres
[' ', ',', '.', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', 'ñ', '‚']


##P2. pasar el texto a números

---
as redes neuronales solo procesan valores numéricos, no letras, por tanto tenemos que traducir los caracteres a representación numérica. Para ello crearemos dos “tablas de traducción”: una de caracteres a números y otra de números a caracteres

In [7]:
char2idx = {u:i for i, u in enumerate(vocab)} # asignamos un número a cada vocablo
idx2char = np.array(vocab)
#-----------revisando las conversiones
#for char,_ in zip(char2idx, range(len(vocab))):
#    print(' {:4s}: {:3d},'.format(repr(char),char2idx[char]))

#pasamos todo el texto a números
texto_como_entero= np.array([char2idx[c] for c in texto])
print('texto: {}'.format(repr(texto[:100])))
print('{}'.format(repr(texto_como_entero[:100])))

texto: 'los musulmanes shiitas de el salvador,atraves de la biblioteca islamica fatima hazzahra,hemos realiz'
array([14, 17, 21,  0, 15, 23, 21, 23, 14, 15,  3, 16,  7, 21,  0, 21, 10,
       11, 11, 22,  3, 21,  0,  6,  7,  0,  7, 14,  0, 21,  3, 14, 24,  3,
        6, 17, 20,  1,  3, 22, 20,  3, 24,  7, 21,  0,  6,  7,  0, 14,  3,
        0,  4, 11,  4, 14, 11, 17, 22,  7,  5,  3,  0, 11, 21, 14,  3, 15,
       11,  5,  3,  0,  8,  3, 22, 11, 15,  3,  0, 10,  3, 28, 28,  3, 10,
       20,  3,  1, 10,  7, 15, 17, 21,  0, 20,  7,  3, 14, 11, 28])


#P2.1 Exportar vocablos y matriz de numerica

In [8]:
rows=[]
columns=['num','vocab']
for i, voc in enumerate(vocab):
  #print(i,'-->', voc)
  rows.append([i,voc])
df= pd.DataFrame(columns=['num','vocab'],data=rows)
df.head(10)
df.to_csv('coran_data_vocab.csv',index=False)

##P3. preparar los datos para ser usados en la RNN

In [9]:
char_dataset= tf.data.Dataset.from_tensor_slices(texto_como_entero)
#cantidad de secuencia de caracteres
secu_length=300
#creamos secuencias de maximo 100 caractereres
secuencias= char_dataset.batch(secu_length+1, drop_remainder=True)
for item in secuencias.take(10):
  print(repr(''.join(idx2char[item.numpy()])))

'los musulmanes shiitas de el salvador,atraves de la biblioteca islamica fatima hazzahra,hemos realizado con mucho amor y dedicacion para el engrandecimiento del islam,la presente edicion del coran,libro bendito,que constituye la culminacion de los mensajes que ala ha enviado a traves de todas las epo'
'cas a la humanidad,por medio de sus mensajeros y profetas.en el se encuentra la contestacion a nuestras mas intrinsecas interrogantes,los diversos topicos de interes para el ser humano,las enseñanzas que nos guian hacia el dios unico alabado y exaltado sea.de tan vital importancia es para los creyent'
'es y las creyentes,pues constituye su primera fuente de consulta.sin embargo,acontece que en la mayoria de los paises latinoamericanos es dificil adquirir una version en nuestra lengua de tan generoso texto,el cual es indubitablemente la escritura por excelencia,el libro que ha dado origen a millones'
' de libros mas.es por ello,que nuestro objetivo primordial ha sido proporcionar al lect

###P3.1 separar los datos en agrupamientos (batches)

In [10]:
#funcion para obtener el conjunto de datos de trainning
def split_input_target(chunk):
  input_text = chunk[:-1]
  target_text= chunk[1:]
  return input_text, target_text

dataset  = secuencias.map(split_input_target)
#el dataset contiene un conjunto de parejas de secuencia de texto
#(con la representación numérica de los caracteres), donde el 
#primer componente de la pareja contiene un paquete con una secuencia 
#de 100 caracteres del texto original y la segunda su correspondiente salida, 
#también de 100 caracteres. )
for input_example, target_example in dataset.take(1):
  print('input data: ', repr(''.join(idx2char[input_example.numpy()])))
  print('Target data: ', repr(''.join(idx2char[target_example.numpy()])))

input data:  'los musulmanes shiitas de el salvador,atraves de la biblioteca islamica fatima hazzahra,hemos realizado con mucho amor y dedicacion para el engrandecimiento del islam,la presente edicion del coran,libro bendito,que constituye la culminacion de los mensajes que ala ha enviado a traves de todas las ep'
Target data:  'os musulmanes shiitas de el salvador,atraves de la biblioteca islamica fatima hazzahra,hemos realizado con mucho amor y dedicacion para el engrandecimiento del islam,la presente edicion del coran,libro bendito,que constituye la culminacion de los mensajes que ala ha enviado a traves de todas las epo'


In [11]:
#imprimimos el tensor del dataset
print(dataset)
#Hyper-Parametros para entrenamiento  de una rede neuronal 
#   -los datos se agrupan en batch
BATCH_SIZE= 128
#    -Tamaño de memoria disponible 
BUFFER_SIZE=10000
dataset= dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)
print (dataset)

<MapDataset shapes: ((300,), (300,)), types: (tf.int64, tf.int64)>
<BatchDataset shapes: ((128, 300), (128, 300)), types: (tf.int64, tf.int64)>


##P4.Construcción del modelo RNN

---
Para construir el modelo usaremos tf.keras.Sequential. Usaremos una versión mínima de RNN, que contenga solo una capa LSTM y 3 capas.


In [12]:
#como es un problema de clasificación estándar 
#para el que debemos definir la función de Lossy el optimizador.
def lossy(labels, logits):
  return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

def create_model(vocab_size, embedding_dim, rnn_units, batch_size):
  #creando el modelo
  model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim,
                              batch_input_shape=[batch_size, None]),
    tf.keras.layers.LSTM(rnn_units,
                         return_sequences=True,
                         stateful=True,
                         recurrent_initializer='glorot_uniform'),
    tf.keras.layers.Dense(vocab_size)                               
  ])
  #En cuanto al optimizador usaremos tf.keras.optimizers.Adam 
  #con los argumentos por defecto del optimizador Adam. 
  model.compile(optimizer='rmsprop',
              loss=lossy,
              metrics=['accuracy'])
  return model
vocab_size= len(vocab)
#dimensiones de los vectores que tendrá la capa.
embedding_dim= 1024
#cantidad de neuronas
rnn_units=1024
#creamos nuestra red neuronal RNN
model=create_model(vocab_size   =vocab_size,
                  embedding_dim =embedding_dim,
                  rnn_units     =rnn_units,
                  batch_size    =BATCH_SIZE)
#summary()para visualizar la estructura del modelo
model.summary()
#resultados=  -La capa LSTM consta más de 5 millones de parametros)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (128, None, 1024)         31744     
                                                                 
 lstm (LSTM)                 (128, None, 1024)         8392704   
                                                                 
 dense (Dense)               (128, None, 31)           31775     
                                                                 
Total params: 8,456,223
Trainable params: 8,456,223
Non-trainable params: 0
_________________________________________________________________


###P4.1 Creando chekpoints

---
una técnica de tolerancia de fallos para procesos cuyo tiempo de ejecución es muy largo. La idea es guardar una instantánea del estado del sistema periódicamente para recuperar desde ese punto la ejecución en caso de fallo del sistema.

---
los crearemos en google drive para mejorar la capacidad de reentrenamiento de la red


In [13]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [14]:
checkpoint_dir='/content/gdrive/MyDrive/Colab Notebooks/checkpoints_proyecto_final/'
checkpoint_prefix= os.path.join(checkpoint_dir,"cp_{epoch:04d}.ckpt")

cp_callback=tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_prefix,
                                               monitor='loss',
                                               verbose=1,
                                               save_weights_only=True,
                                               save_best_only=True,
                                               mode='auto')

###P4.2 entrenando

####P4.2a entrenando para usar chekpoints

In [ ]:
EPOCHS=50
history=model.fit(dataset, 
                  epochs=EPOCHS, 
                  verbose=1,
                  callbacks=[cp_callback])

Epoch 1/50
19/19 [==============================] - ETA: 0s - loss: 2.9915 - accuracy: 0.1969
Epoch 00001: loss improved from inf to 2.99146, saving model to /content/gdrive/MyDrive/Colab Notebooks/checkpoints_proyecto_final/cp_0001.ckpt
19/19 [==============================] - 22s 1s/step - loss: 2.9915 - accuracy: 0.1969
Epoch 2/50
19/19 [==============================] - ETA: 0s - loss: 2.4280 - accuracy: 0.2519
Epoch 00002: loss improved from 2.99146 to 2.42795, saving model to /content/gdrive/MyDrive/Colab Notebooks/checkpoints_proyecto_final/cp_0002.ckpt
19/19 [==============================] - 22s 1s/step - loss: 2.4280 - accuracy: 0.2519
Epoch 3/50
19/19 [==============================] - ETA: 0s - loss: 2.2845 - accuracy: 0.2808
Epoch 00003: loss improved from 2.42795 to 2.28450, saving model to /content/gdrive/MyDrive/Colab Notebooks/checkpoints_proyecto_final/cp_0003.ckpt
19/19 [==============================] - 22s 1s/step - loss: 2.2845 - accuracy: 0.2808
Epoch 4/50
19/19 


#####4.2a-1 entrenando desde un checkpoint

---
Desde la carpeta que optamos guardar los checkpoints

*   el archivo .data es el archivo que contiene nuestras variables de entrenamiento y vamos a ir tras él.
*   el archivo checkpoint, simplemente mantiene un registro de los últimos archivos de punto de control guardados




In [15]:
#creamos un modelo con iguales caracteristicas al 1° modelo
model2=create_model(vocab_size   =vocab_size,
                  embedding_dim =embedding_dim,
                  rnn_units     =rnn_units,
                  batch_size    =BATCH_SIZE)

#buscamos el ultimo checkpoint de entrenamiento
latest = tf.train.latest_checkpoint(checkpoint_dir)
print(latest)

/content/gdrive/MyDrive/Colab Notebooks/checkpoints_proyecto_final/cp_0349.ckpt


In [17]:
# cargamos los pesos al nuevo modelo (estos valores tienes una variación de un 10%)
model2.load_weights(latest)
# continuamos el entrenamiento desde el checkpoint en que quedamos
history_V2=model2.fit(dataset, 
                    epochs=200, 
                    verbose=1,
                    callbacks=[cp_callback])

Epoch 1/200
19/19 [==============================] - ETA: 0s - loss: 0.0749 - accuracy: 0.9863
Epoch 00001: loss did not improve from 0.06231
19/19 [==============================] - 21s 1s/step - loss: 0.0749 - accuracy: 0.9863
Epoch 2/200
19/19 [==============================] - ETA: 0s - loss: 0.0806 - accuracy: 0.9841
Epoch 00002: loss did not improve from 0.06231
19/19 [==============================] - 21s 1s/step - loss: 0.0806 - accuracy: 0.9841
Epoch 3/200
19/19 [==============================] - ETA: 0s - loss: 0.0681 - accuracy: 0.9872
Epoch 00003: loss did not improve from 0.06231
19/19 [==============================] - 21s 1s/step - loss: 0.0681 - accuracy: 0.9872
Epoch 4/200
19/19 [==============================] - ETA: 0s - loss: 0.0774 - accuracy: 0.9857
Epoch 00004: loss did not improve from 0.06231
19/19 [==============================] - 21s 1s/step - loss: 0.0774 - accuracy: 0.9857
Epoch 5/200
19/19 [==============================] - ETA: 0s - loss: 0.0688 - accura

##P5. Generando texto nuevo usando la RNN

In [18]:
#creamos un modelo tomando como base el ultimo checkpoint
model = create_model(vocab_size, embedding_dim, rnn_units, batch_size=1)
model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))##tomamos el ultimo checkpoint
model.build(tf.TensorShape([1,None]))

In [19]:
#funcion para generar texto
def generate_text(model, start_string):
  #definimos cuantos tensores/cantidad de texto generaremos
  num_generate=500
  #convertimos el texto en números
  input_eval=[char2idx[s] for s in start_string]
  input_eval= tf.expand_dims (input_eval,0)
  text_generated = []

  temperature = 0.2  #(0.0 a  1) entre más alta la temperatura más creatividad al modelo, pero tambien más errores ortograficos.
  model.reset_states() #bucle para generar caracteres, mediante predicciones
  for i in range(num_generate):
    predictions = model(input_eval)
    predictions = tf.squeeze(predictions, 0)
    predictions = predictions / temperature
    predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()
    input_eval= tf.expand_dims([predicted_id],0)
    text_generated.append (idx2char[predicted_id])
  
  return (start_string+ ''.join(text_generated))

###P5.1 generando texto 

In [20]:
print(generate_text(model, start_string=u"los musulmanes"))

los musulmanes shiitas de el salvador,atraves de la biblioteca islamica fatima hazzahra,hemos realizado con mucho amor y dedicacion para el engrandecimiento del islam,la presente edicion del coran,libro bendito,que constituye la culminacion de los mensajes que ala ha enviado a traves de todas las epostabais a sus tropas y las aguas del mar les cubrieron.faraon habia extraviado a su pueblo,no le habia dirigido bien.hijos de israel os hemos salvado de vuestros enemigos y nos hemos dado cita con vosotros en la l


##P6.exportando modelo

---
Guardamos y Serializamos el Modelo (con esto ya podemos vender nuestro modelo de predicción de texto según lo aprendido por nuestra RNN).


In [21]:
from keras.models import model_from_json
import os
dir_export= '/content/gdrive/MyDrive/Colab Notebooks//Proyecto_Final/'
#dir_export= os.path.join(dir_drive)
# Serializamos el modelo en forma JSON
model_json = model.to_json()
with open(os.path.join(dir_export,'RNN_Coran_json.json'), 'w') as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights(os.path.join(dir_export,'RNN_Coran_pesos.hdf5'))
model.save(os.path.join(dir_export,'RNN_Coran_model.h5'))
print("Modelo salvado en Drive de google")

Modelo salvado en Drive de google
